In [1058]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_rows',5)
pd.set_option('display.max_columns',None)

In [1059]:
train = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Flight_Price_RandomForest\Flight_Ticket_Participant_Datasets\Data_Train.xlsx')

In [1060]:
test = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Flight_Price_RandomForest\Flight_Ticket_Participant_Datasets\Test_set.xlsx')

In [1061]:
submission = pd.read_excel(r'E:\INCEPTEZ\Training\Basics_python\Flight_Price_RandomForest\Flight_Ticket_Participant_Datasets\Sample_submission.xlsx')

In [1062]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [1063]:
test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [1064]:
submission.head()

,Price
0,15998
1,16612
2,25572
3,25778
4,16934


In [1065]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [1066]:
train.describe(include='object').T

,count,unique,top,freq
Airline,10683,12,Jet Airways,3849
Date_of_Journey,10683,44,18/05/2019,504
...,...,...,...,...
Total_Stops,10682,5,1 stop,5625
Additional_Info,10683,10,No info,8345


In [1067]:
train['source']='train'
test['source']='test'

In [1068]:
train.shape , test.shape

((10683, 12), (2671, 11))

In [1069]:
data = pd.concat([train,test])

In [1070]:
data = data.reset_index(drop=True)

In [1071]:
data.shape

(13354, 12)

In [1072]:
display(data.head(1), data.tail(1))

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,train


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source
13353,Multiple carriers,15/06/2019,Delhi,Cochin,DEL → BOM → COK,04:55,19:15,14h 20m,1 stop,No info,NaN,test


### Data Cleaning

In [1073]:
data['Airline'].value_counts()

Jet Airways                4746
IndiGo                     2564
                           ... 
Vistara Premium economy       5
Trujet                        1
Name: Airline, Length: 12, dtype: int64

In [1074]:
data['Date_of_Journey'].value_counts()


18/05/2019    633
6/06/2019     630
             ... 
12/04/2019     74
1/03/2019      59
Name: Date_of_Journey, Length: 44, dtype: int64

In [1075]:
data['Date_of_Journey'] = pd.to_datetime(data['Date_of_Journey'])

In [1077]:
data['Day']= data['Date_of_Journey'].dt.day
data['Year']=data['Date_of_Journey'].dt.year
data['Month']= data['Date_of_Journey'].dt.month

In [1078]:
data =data.drop(columns=['Date_of_Journey'])

In [1079]:
data['Source'].value_counts()

Delhi       5682
Kolkata     3581
Banglore    2752
Mumbai       883
Chennai      456
Name: Source, dtype: int64

In [1080]:
data['Destination'].value_counts()

Cochin       5682
Banglore     3581
             ... 
Hyderabad     883
Kolkata       456
Name: Destination, Length: 6, dtype: int64

In [1081]:
data = pd.get_dummies(data , columns=['Airline','Source','Destination'])

In [1083]:
data['Route'].value_counts()


DEL → BOM → COK                3000
BLR → DEL                      1941
                               ... 
BOM → JDH → JAI → DEL → HYD       1
BOM → IXC → DEL → HYD             1
Name: Route, Length: 132, dtype: int64

In [1084]:
data['Route'] =data['Route'].apply(str)

In [1085]:
data['Route'].apply(lambda x: re.sub('[^A-Za-z]+',',',(x)))

0                BLR,DEL
1        CCU,IXR,BBI,BLR
              ...       
13352        DEL,BOM,COK
13353        DEL,BOM,COK
Name: Route, Length: 13354, dtype: object

In [1086]:
#data['Arrival'] = data['Route'].apply(lambda x:(x.split(',')[-1]))
#data['Departure'] = data['Route'].apply(lambda x:(x.split(',')[0]))

In [1088]:
data.head()

,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,source,Day,Year,Month,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,Airline_Vistara,Airline_Vistara Premium economy,Source_Banglore,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897.0,train,24,2019,3,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
1,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662.0,train,5,2019,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
2,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882.0,train,6,2019,9,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218.0,train,5,2019,12,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0
4,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302.0,train,3,2019,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [1089]:
data['Duration'] =data['Duration'].apply(lambda x: re.sub('[^0-9]+','.',x))

In [1090]:
data['Duration'] =data['Duration'].apply(lambda x: '{0:.1}'.format(x))

In [1091]:
data['Duration'] =data['Duration'].astype(int)

In [1093]:
data['Total_Stops'].value_counts()

1 stop      7056
non-stop    4340
2 stops     1899
3 stops       56
4 stops        2
Name: Total_Stops, dtype: int64

In [1094]:
data['Total_Stops'] =data['Total_Stops'].replace({'1 stop':1,'non-stop':0,'2 stops':2,'3 stops':3,'4 stops':4})

In [1095]:
data['Total_Stops'] =data['Total_Stops'].fillna(0)

In [1096]:
data['Total_Stops'] =data['Total_Stops'].astype(int)

In [1098]:
data['Additional_Info'] =[0 if x== 'No info' else 1 for x in data['Additional_Info']]

In [1099]:
data['Additional_Info'].value_counts()

0    10493
1     2861
Name: Additional_Info, dtype: int64

In [1100]:
data['Route'] = data['Route'].map(data['Route'].value_counts())

In [1101]:
pd.set_option('display.max_columns',5)
data['Arrival_Time'].value_counts()

19:00           536
21:00           466
               ... 
00:25 22 Jun      1
07:45 13 May      1
Name: Arrival_Time, Length: 1451, dtype: int64

In [1102]:
data['Arrival_Time'] = data['Arrival_Time'].apply(lambda x: re.sub('[^0-9]+','.',x))

In [1103]:
data['Arrival_Time'] =data['Arrival_Time'].apply(lambda x: '{0:.5}'.format(x))

In [1104]:
data['Arrival_Time'] =data['Arrival_Time'].astype(float)

In [1105]:
data['Dep_Time'] =data['Dep_Time'].apply(lambda x: re.sub('[^0-9]+','.',x))

In [1106]:
data['Dep_Time'] =data['Dep_Time'].apply(lambda x: '{0:.5}'.format(x))

In [1107]:
data['Dep_Time'] =data['Dep_Time'].astype(float)

In [1108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 34 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Route                                      13354 non-null  int64  
 1   Dep_Time                                   13354 non-null  float64
 2   Arrival_Time                               13354 non-null  float64
 3   Duration                                   13354 non-null  int32  
 4   Total_Stops                                13354 non-null  int32  
 5   Additional_Info                            13354 non-null  int64  
 6   Price                                      10683 non-null  float64
 7   source                                     13354 non-null  object 
 8   Day                                        13354 non-null  int64  
 9   Year                                       13354 non-null  int64  
 10  Month                 

In [1109]:
data.isna().sum()

Route                    0
Dep_Time                 0
                        ..
Destination_Kolkata      0
Destination_New Delhi    0
Length: 34, dtype: int64

In [1110]:
train_preprocessed = data[data['source']=='train']

In [1111]:
test_preprocessed = data[data['source']=='test']

In [1112]:
train_preprocessed=train_preprocessed.drop(columns=['source'])

In [1113]:
test_preprocessed=test_preprocessed.drop(columns=['source','Price'])

In [1114]:
train_preprocessed.head()

,Route,Dep_Time,...,Destination_Kolkata,Destination_New Delhi
0,1941,22.20,...,0,1
1,7,5.50,...,0,0
2,46,9.25,...,0,0
3,11,18.05,...,0,0
4,4,16.50,...,0,1


In [1115]:
test_preprocessed.head()

,Route,Dep_Time,...,Destination_Kolkata,Destination_New Delhi
10683,3000,17.30,...,0,0
10684,78,6.20,...,0,0
10685,3000,19.15,...,0,0
10686,3000,8.00,...,0,0
10687,1941,23.55,...,0,0


In [1116]:
test_preprocessed.reset_index(drop=True, inplace =True)

In [1117]:
train_preprocessed.shape , test_preprocessed.shape

((10683, 33), (2671, 32))

In [1118]:
train_preprocessed.to_csv('train_preprocessed.csv', index= False)
test_preprocessed.to_csv('test_preprocessed.csv', index= False)